In [6]:
search_for = 113
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00788259506225586
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 164503141
type: [1, 1, 1, 1, 113] 113
cases of this type: 163047361
10000 10.967740435770056
20000 10.871476359286287
30000 11.20073561626276
40000 15.431965803683022
50000 49.936083272192555
60000 41.460853946624965
70000 43.26242063535893
80000 39.651947446895775
90000 40.54062395391394
100000 43.499273302887225
110000 39.4585361415449
120000 50.53279957529195
130000 42.797481395955145
140000 51.210715602527976
150000 48.540693676815515
160000 48.982922620483876
170000 54.403473218390694
180000 60.30646275542527
190000 57.634656284036176
200000 61.599030466799356
210000 69.12045404200867
220000 55.665532134229565
230000 56.695530379958754
240000 59.63864838042891
250000 62.35109959932632
260000 60.21988011515762
270000 60.83263385423879
280000 57.33877647272964
290000 64.11439570558231
300000 78.30798336648805
310000 77.63846218

3030000 117.83198394594653
3040000 112.20303190219502
3050000 124.61262569605145
3060000 108.38875671327172
3070000 118.62226638347893
3080000 111.03807278673153
3090000 118.6143220205111
3100000 114.98972143987355
3110000 109.54833797249056
3120000 112.41182419658965
3130000 107.90725253000898
3140000 112.97478355544105
3150000 107.80025354129637
3160000 115.819239280537
3170000 113.48269875932142
3180000 111.73650668196436
3190000 118.89218140495385
3200000 117.24806142713902
3210000 107.68929010286739
3220000 116.85524878627707
3230000 117.58937205407945
3240000 118.55574542555597
3250000 127.51561166078386
3260000 125.67117269870413
3270000 112.3635687298744
3280000 129.01077751364573
3290000 137.05311649770456
3300000 130.55289460785696
3310000 109.5723696882301
3320000 110.58050587703599
3330000 114.37903751477839
3340000 108.81966873232389
3350000 107.07555284896789
3360000 109.43700867329142
3370000 108.37878736389503
3380000 105.74371023428759
3390000 107.44210900254745
340000

6090000 116.30355214665379
6100000 113.99574417309807
6110000 118.84609502437786
6120000 117.26894447371303
6130000 115.76550490271123
6140000 114.21184379851583
6150000 122.05502822770633
6160000 114.94054480333291
6170000 115.16214467001143
6180000 123.71167216470728
6190000 110.77469570048065
6200000 109.72335344758632
6210000 107.39817719926836
6220000 111.52538544221139
6230000 109.83206195680054
6240000 108.70041042029969
6250000 109.52468606749214
6260000 105.14017321023857
6270000 108.06067708966059
6280000 110.65582654209291
6290000 107.37778401979078
6300000 107.06365469511749
6310000 105.33907742784855
6320000 112.64745417701961
6330000 106.24292850891513
6340000 113.44932713046796
6350000 117.1755406576485
6360000 109.61088504794762
6370000 118.67589101404184
6380000 107.99684891288763
6390000 123.29836526063347
6400000 109.64713061850297
6410000 110.44661368882846
6420000 109.5938365933898
6430000 97.3867223550921
6440000 115.89426292924388
6450000 107.44157664430132
64600

9150000 104.08706490685847
9160000 101.81642821348053
9170000 102.29837288755317
9180000 108.8612265534359
9190000 105.93814685462627
9200000 111.15300936893783
9210000 110.48388428451061
9220000 108.66570970530992
9230000 111.7595595807465
9240000 104.71901465203683
9250000 116.4506744801265
9260000 104.7898962110693
9270000 112.20592388002835
9280000 105.73269930895195
9290000 98.23717436280305
9300000 109.31680526514461
9310000 96.33618905314562
9320000 105.08285967706074
9330000 96.3790627525842
9340000 121.32307962624498
9350000 82.57605774925001
9360000 14.494866852925917
9370000 14.428703571068286
9380000 15.296751052851066
9390000 14.045344751496353
9400000 14.071025446045718
9410000 14.24928763189967
9420000 125.24566768451312
9430000 114.25814320081645
9440000 101.6962929932496
9450000 89.30175061637826
9460000 108.02409034360745
9470000 95.42243416439148
9480000 109.73378321051854
9490000 97.90966957387096
9500000 108.33485252685894
9510000 109.40223579336603
9520000 108.229

12150000 96.58632325922063
12160000 106.92657223093707
12170000 98.56840531836929
12180000 103.36074746844572
12190000 89.1668249970687
12200000 98.20376626750848
12210000 86.21863308588304
12220000 100.25059510773401
12230000 122.13295001958592
12240000 38.204452204604735
12250000 13.91061926007684
12260000 13.683189469752744
12270000 13.63497585965246
12280000 13.660717112826486
12290000 14.080677924667246
12300000 52.40589692502112
12310000 133.18678238623025
12320000 107.09636999869105
12330000 83.11961770417496
12340000 96.91120314295948
12350000 84.74967156086346
12360000 99.73597854356468
12370000 98.17577035425543
12380000 99.21720355748386
12390000 103.54429818972638
12400000 94.05523103573258
12410000 101.93535289594558
12420000 93.05364460244854
12430000 103.45995702741682
12440000 98.18399348786554
12450000 104.43106140964022
12460000 100.46836959585451
12470000 100.45799800876972
12480000 101.82311190534233
12490000 104.06243104987072
12500000 101.88424449373413
12510000 1

...

151730000 8.851150048559042
151740000 9.059774817098479
151750000 8.26755123852684
151760000 8.995170560355657
151770000 8.731397780842576
151780000 8.742321280071776
151790000 8.719058506435832
151800000 8.81121693059906
151810000 8.775571280255555
151820000 9.351362914715217
151830000 9.118660739558823
151840000 9.100689535547037
151850000 8.775496241759544
151860000 8.754911624406994
151870000 9.103709081676246
151880000 8.554222998000357
151890000 8.69821479704884
151900000 8.75966576321304
151910000 8.525915658100736
151920000 8.639417561693218
151930000 9.077013372044226
151940000 9.297713569885282
151950000 8.838657177860254
151960000 8.30152853714788
151970000 8.576213806827566
151980000 8.185592857851697
151990000 8.605909844133974
152000000 8.674773528541266
152010000 8.596414242338724
152020000 8.673069095730245
152030000 8.60774022409559
152040000 8.539468559833606
152050000 8.971068372353335
152060000 8.691934046076689
152070000 9.420001365330279
152080000 8.06229001238191

154660000 6.790624446115095
154670000 7.111730124766463
154680000 6.710267164670951
154690000 6.813886813892364
154700000 6.925146139827715
154710000 7.567634013813344
154720000 7.679710480320713
154730000 7.9312802934904365
154740000 7.2529314798660085
154750000 7.7585768390375565
154760000 6.430821563820203
154770000 6.85787835563102
154780000 7.205913166239679
154790000 6.5827247901997366
154800000 7.3593776692874435
154810000 7.291078148476282
154820000 7.310369064053085
154830000 6.514958877616267
154840000 6.432393737472832
154850000 6.914888434584485
154860000 6.482548076440718
154870000 6.87754667659986
154880000 6.195479894710124
154890000 6.477445293771128
154900000 6.893898956177295
154910000 6.1402459396559
154920000 6.773851519418339
154930000 6.724971900129518
154940000 6.822120709928904
154950000 7.162049689754486
154960000 6.38911046238184
154970000 7.609746524497503
154980000 6.705428681969477
154990000 7.318658812524319
155000000 6.981374245960281
155010000 7.00585075

157580000 4.999172832399037
157590000 5.18090194191436
157600000 4.973750206320345
157610000 5.49958293445915
157620000 5.34244389369162
157630000 5.300694301670075
157640000 4.928042599198633
157650000 4.995203701791074
157660000 4.731166755872249
157670000 5.1185319414138
157680000 5.262428176493618
157690000 4.870833104708949
157700000 4.758107271147648
157710000 4.608730895554351
157720000 4.875465880932212
157730000 4.5599742930530835
157740000 4.536702780287637
157750000 4.696084157624006
157760000 4.656938425876915
157770000 4.609036695859445
157780000 4.680768400274396
157790000 4.641240088006059
157800000 4.475480457583441
157810000 4.924036150274614
157820000 4.495299623712746
157830000 4.58690147625304
157840000 4.558814957694054
157850000 4.654876905164103
157860000 4.722304235753616
157870000 4.690148862131179
157880000 4.882453318959442
157890000 4.646332441735187
157900000 5.050498634703656
157910000 4.631980823007438
157920000 4.998523766607013
157930000 4.4602263630501

160470000 3.221500447526905
160480000 3.0942780905697544
160490000 2.9875108692350585
160500000 3.1569596870489387
160510000 3.218504759397487
160520000 3.3600319028208387
160530000 2.987070926210834
160540000 3.026575407772243
160550000 3.066495638811356
160560000 3.2063275069061854
160570000 2.860318417690277
160580000 2.865124236820141
160590000 2.887086982561827
160600000 2.872301317667961
160610000 3.118882927486963
160620000 2.8777111346893904
160630000 2.795817306031048
160640000 2.799901250193755
160650000 2.7359890385173227
160660000 2.6415717735676965
160670000 2.8375056173518827
160680000 2.642794702390631
160690000 2.7762658953456683
160700000 2.7060429588465955
160710000 2.8506500799322456
160720000 2.6278756925840976
160730000 2.6980463259671135
160740000 2.849478088844114
160750000 2.677803204119484
160760000 2.9400209762147003
160770000 2.5615398220608365
160780000 2.7537491589762566
160790000 2.6290779739471413
160800000 2.7132267514773147
160810000 2.732382786085188
1

163300000 0.08482003371812899
163310000 0.08434796065398721
163320000 0.08296056600333584
163330000 0.08306044918847084
163340000 0.0817739228720665
163350000 0.0807046871303055
163360000 0.08101815259116889
163370000 0.08113587495707804
163380000 0.07983490319579839
163390000 0.07773188972292344
163400000 0.07796045249114433
163410000 0.0771258300736679
163420000 0.07647502668631077
163430000 0.07737189924461974
163440000 0.0747353512503902
163450000 0.07360489188764494
163460000 0.07369210306957033
163470000 0.07251912865564558
163480000 0.07237987861982981
163490000 0.07123532919222117
163500000 0.07089536471858952
163510000 0.06929427421706916
163520000 0.06898978927905693
163530000 0.06796445817425516
163540000 0.06750580878243845
163550000 0.0666775132680999
163560000 0.06603925763070584
163570000 0.06519429294908047
163580000 0.06453443025622765
163590000 0.06375278330774439
163600000 0.06319559156078101
163610000 0.06406502561973863
163620000 0.0632591626696388
163630000 0.0616